# Speech Emotion Recognition Demo 2

## Importing packages

In [ ]:
!pip install speechbrain
!pip install transformers
!git clone https://github.com/GasserElbanna/serab-byols.git
!python3 -m pip install -e ./serab-byols

!pip install tqdm==4.60.0
!pip install opensmile
!git clone https://github.com/satvik-dixit/speech_emotion_recognition.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 496 kB 6.7 MB/s 
     |████████████████████████████████| 750.6 MB 9.2 kB/s 
     |████████████████████████████████| 101 kB 13.0 MB/s 
     |████████████████████████████████| 1.2 MB 28.4 MB/s 
     |████████████████████████████████| 596 kB 46.1 MB/s 
     |████████████████████████████████| 109 kB 56.9 MB/s 
     |████████████████████████████████| 546 kB 52.0 MB/s 
     |████████████████████████████████| 3.7 MB 38.5 MB/s 
     |████████████████████████████████| 2.9 MB 45.0 MB/s 
  Created wheel for hyperpyyaml: filename=HyperPyYAML-1.0.1-py3-none-any.whl size=15192 sha256=8ef0e3d29bda7d4f64d8e4540cd43bc20c8c49f73c1fe1bd4460009e8342df34
  Stored in directory: /root/.cache/pip/wheels/6c/87/65/266d722c3932f81f16332ce842e972be8421e3a9cd3771766b
Successfully built hyperpyyaml
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0+cu113

In [ ]:
import os
import numpy as np
from tqdm import tqdm
from glob import glob
from random import sample

import librosa
import soundfile as sf

import torch
import opensmile
import serab_byols
from transformers import Wav2Vec2Model, HubertModel

from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV, train_test_split

import warnings
warnings.filterwarnings('ignore')


/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)


# Phase 1: Loading and resampling audio files

## Defining a function for loading and resampling audio files

In [ ]:
# Defining a function for loading and resampling audio files

def load_audio_files(audio_files, resampling_frequency=16000, audio_list=None):
  '''
  Loads and resamples audio files 
  
  Parameters
  ------------
  audio_files: string
      The paths of the wav files 
  resampling_frequency: integer
      The frequency which all audios will be resampled to
  audio_list: list 
      The list of torch tensors of audios to which more audios need too be added, empty by default

  Returns
  ------------
  audio_list: list
      A list of torch tensors, one array for each audio file

  '''
  # Making audio_list
  if audio_list is None:
    audio_list = []

  # Resampling
  for audio in audio_files:
    signal, fs = librosa.load(audio, sr=resampling_frequency)
    audio_list.append(torch.from_numpy(signal))
      
  return audio_list
        

## Loading and resampling audiofiles and collecting metadata on EmoDB dataset

In [ ]:
# Phase_1
# Load dataset
! wget http://emodb.bilderbar.info/download/download.zip
! unzip -q download.zip

# Resample dataset
audio_files = glob('/content/wav/*.wav')
audio_list = load_audio_files(audio_files, resampling_frequency=16000)

# Extracting metadata (speakers and labels)
labels = np.array(list(map(lambda x: os.path.basename(x).split('.')[0][-2], audio_files)))
speakers = np.array(list(map(lambda x: os.path.basename(x)[:2], audio_files)))

# ---------------------------------------------------------------------------------------------------

# Verify phase_1
print('Number of audio files: {}'.format(len(audio_list)))
print('Number of speaker classes: {}'.format(len(set(speakers))))
print('Speaker classes: {}'.format(set(speakers)))
print('Number of speakers: {}'.format(len(speakers)))
print('Speakers:')
print(speakers)
print('Number of label classes: {}'.format(len(set(labels))))
print('Label classes: {}'.format(set(labels)))
print('Number of labels: {}'.format(len(labels)))
print('Labels:')
print(labels)

berlin-database-of-emotional-speech-emodb.zip: Skipping, found more recently modified local copy (use --force to force download)
replace wav/03a01Fa.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: All
Number of audio files: 535
Number of speaker classes: 10
Speaker classes: {'13', '15', '12', '08', '14', '16', '09', '10', '03', '11'}
Number of speakers: 535
Speakers:
['15' '08' '13' '13' '13' '11' '13' '13' '09' '08' '16' '15' '12' '08'
 '13' '08' '13' '14' '16' '15' '03' '15' '12' '16' '12' '16' '03' '16'
 '13' '16' '11' '12' '16' '13' '03' '03' '14' '15' '16' '13' '12' '10'
 '14' '14' '09' '16' '03' '16' '03' '08' '15' '14' '16' '16' '11' '03'
 '13' '10' '11' '09' '03' '13' '10' '08' '11' '15' '03' '03' '03' '13'
 '10' '16' '13' '09' '13' '14' '10' '11' '13' '11' '13' '16' '16' '14'
 '09' '09' '09' '03' '12' '15' '08' '03' '12' '11' '12' '16' '15' '03'
 '10' '14' '13' '16' '13' '08' '12' '13' '10' '13' '08' '08' '03' '14'
 '12' '16' '15' '09' '13' '16' '13' '14' '09' '14' '10' '13' '15' '

# Phase 2: Embedding Extraction

## Audio embeddings extraction functions

In [ ]:
# Defining a function for generating audio embedding extraction models

def audio_embeddings_model(model_name):
  '''
  Generates model for embedding extraction 
  
  Parameters
  ------------
  mode_name: string
      The model to used, could be 'wav2vec', 'hubert', 'hybrid_byols', 'compare' or 'egemaps'

  Returns
  ------------
  model: object
      The embedding extraction model
  '''
  if model_name=='wav2vec2':
    model_hub = 'facebook/wav2vec2-large-960h-lv60-self'
    model = Wav2Vec2Model.from_pretrained(model_hub)
  elif model_name=='hubert':
    model_hub = 'facebook/hubert-xlarge-ll60k'
    model = HubertModel.from_pretrained(model_hub)
  elif model_name=='hybrid_byols':
    model_name = 'cvt'
    checkpoint_path = "serab-byols/checkpoints/cvt_s1-d1-e64_s2-d1-e256_s3-d1-e512_BYOLAs64x96-osandbyolaloss6373-e100-bs256-lr0003-rs42.pth"
    model = serab_byols.load_model(checkpoint_path, model_name)
  elif model_name=='compare':
    model = opensmile.Smile(
        feature_set=opensmile.FeatureSet.ComParE_2016,
        feature_level=opensmile.FeatureLevel.Functionals,
    )
  elif model_name=='egemaps':
    model = opensmile.Smile(
        feature_set=opensmile.FeatureSet.eGeMAPSv02,
        feature_level=opensmile.FeatureLevel.Functionals,
    )
  return model


# Defining a function for embedding exctraction from the audio list

def audio_embeddings(audio_list, model_name, model, sampling_rate=16000):
  '''
  Loads and resamples audio files 
  
  Parameters
  ------------
  audio_list: list
      A list of arrays, one array for each audio file
  model_name: string
      The model to used, could be 'wav2vec', 'hubert', 'hybrid_byols', 'compare' or 'egemaps'
  model: object
      The embedding extraction model generated by audio_embeddings_model function
  n_feats: int
      The number of features of each audio file, 6373 for 'compare' and 88 for 'egemaps'

  Returns
  ------------
  embeddings_array: array
      The array containg embeddings of all audio_files, dimension (number of audio files × n_feats)
      
  '''
  if model_name=='hybrid_byols':
    embeddings_array = serab_byols.get_scene_embeddings(audio_list, model)
  else:
    embeddings_list = []
    for i in tqdm(range(len(audio_list))):
      if model_name=='wav2vec2' or model_name=='hubert':
        embeddings = model(audio_list[i].reshape(1,-1)).last_hidden_state.mean(1)
        embeddings_list.append(embeddings[0])
      elif model_name=='compare' or model_name=='egemaps':
        embeddings = model.process_signal(audio_list[i], sampling_rate)
        embeddings_list.append(torch.tensor(embeddings.values[0], dtype=torch.float32))
    embeddings_array = torch.stack(embeddings_list)
  return embeddings_array


## Audio embeddings extraction on EmoDB

In [ ]:
# Phase_2

# Wav2vec
model = audio_embeddings_model(model_name='wav2vec')
embeddings_array_wav2vec = audio_embeddings(audio_list, model_name='wav2vec', model=model)

# Hubert
model = audio_embeddings_model(model_name='hubert')
embeddings_array_hubert = audio_embeddings(audio_list, model_name='hubert', model=model)

# ---------------------------------------------------------------------------------------------------

# Verify Phase_2
models = ['wav2vec2', 'hubert']
embeddings_arrays = {'wav2vec2': embeddings_array_wav2vec, 'hubert': embeddings_array_hubert}

for model in models:
  print()
  print()
  print('MODEL: {}'.format(model))
  print()
  print('The shape of the embeddings array is {}'.format(embeddings_arrays[model].shape))
  print('The embeddings array is: ')
  print((embeddings_arrays[model]))


In [ ]:
# # Phase_2

# # Hybrid BYOLS
# model = audio_embeddings_model(model_name='hybrid_byols')
# embeddings_array_byols = audio_embeddings(audio_list, model_name='hybrid_byols', model=model)

# # EmoDB compare
# model = audio_embeddings_model(model_name='compare')
# embeddings_array_compare = audio_embeddings(audio_list, model_name='compare', model=model)

# # EmoDB egemaps
# model = audio_embeddings_model(model_name='egemaps')
# embeddings_array_egemaps = audio_embeddings(audio_list, model_name='egemaps', model=model)

# # ---------------------------------------------------------------------------------------------------

# # Verify Phase_2
# models = ['hybrid_byols', 'compare', 'egemaps']
# embeddings_arrays = {'hybrid_byols': embeddings_array_byols, 'compare':embeddings_array_compare, 'egemaps':embeddings_array_egemaps}

# for model in models:
#   print()
#   print()
#   print('MODEL: {}'.format(model))
#   print()
#   print('The shape of the embeddings array is {}'.format(embeddings_arrays[model].shape))
#   print('The embeddings array is: ')
#   print((embeddings_arrays[model]))


100%|██████████| 535/535 [00:57<00:00,  9.25it/s]



MODEL: wav2vec2

The shape of the embeddings array is torch.Size([535, 1024])
The embeddings array is: 
tensor([[-0.0854, -0.1182,  0.0090,  ...,  0.1121, -0.0221,  0.2053],
        [-0.0649, -0.1166, -0.0010,  ...,  0.1056, -0.0498,  0.2029],
        [-0.0660, -0.1250,  0.0148,  ...,  0.1073, -0.0408,  0.2018],
        ...,
        [-0.0661, -0.1327,  0.0590,  ...,  0.1059, -0.0196,  0.1992],
        [-0.0289, -0.1255,  0.0099,  ...,  0.1206, -0.0237,  0.2044],
        [-0.0488, -0.1268,  0.0219,  ...,  0.1213, -0.0271,  0.2042]])


MODEL: hubert

The shape of the embeddings array is torch.Size([535, 1280])
The embeddings array is: 
tensor([[-0.1409,  0.0715, -0.0276,  ...,  0.0794, -0.0020, -0.0408],
        [-0.1844, -0.0501,  0.0135,  ...,  0.0842,  0.0074, -0.0492],
        [-0.0795,  0.0288,  0.0077,  ..., -0.0329, -0.0064,  0.0352],
        ...,
        [-0.1168,  0.0273,  0.0328,  ...,  0.1451, -0.0298, -0.0803],
        [-0.1545,  0.0423,  0.0530,  ..., -0.0081,  0.0088,  0.

# Phase 3: Downstream Task - Speech Emotion Recognotion

## Speaker normalisation functions

In [ ]:
# Defining a function for speaker normalisation using standard scaler

def speaker_normalisation(embeddings_array, speakers):
  '''
  Normalises embeddings_array for each speaker
  
  Parameters
  ------------
  embeddings_array: array
      The array of embeddings, one row for each audio file
  speakers: list 
      The list of speakers

  Returns
  ------------
  embeddings_array: array
      The array containg normalised embeddings of all audio_files, dimension (number of audio files × n_feats)
      
  '''
  speaker_ids = set(speakers)
  for speaker_id in speaker_ids:
    speaker_embeddings_indices = np.where(np.array(speakers)==speaker_id)[0]
    speaker_embeddings = embeddings_array[speaker_embeddings_indices,:]
    scaler = StandardScaler()
    normalised_speaker_embeddings = scaler.fit_transform(speaker_embeddings)
    embeddings_array[speaker_embeddings_indices] = torch.tensor(normalised_speaker_embeddings).float()
  return embeddings_array


## Speaker normalisation on EmoDB

In [ ]:
# Normalised arrays
normalised_embeddings_wav2vec = speaker_normalisation(embeddings_array_wav2vec, speakers)
normalised_embeddings_hubert = speaker_normalisation(embeddings_array_hubert, speakers)

# Verifying normalised_embeddings_arrays

models = ['wav2vec2', 'hubert']
normalised_embeddings_arrays = {'wav2vec2': normalised_embeddings_wav2vec, 'hubert': normalised_embeddings_hubert}

for model in models:
  print()
  print()
  print('MODEL: {}'.format(model))
  print()
  print('The shape of the normalised embeddings array is: {}'.format(normalised_embeddings_arrays[model].shape))
  print('Normalised Embeddings Array:')
  print((normalised_embeddings_arrays[model]))
  print()
  columnwise_mean = torch.mean(normalised_embeddings_arrays[model], 0)
  print('Columnwise_mean:')
  print(columnwise_mean)
  if torch.all(columnwise_mean < 10**(-6)):
    print('All means are less than 10**-6')
  else:
    print('All means are NOT less than 10**-6')

In [ ]:
# # Normalised arrays
# normalised_embeddings_byols = speaker_normalisation(embeddings_array_byols, speakers)
# normalised_embeddings_compare= speaker_normalisation(embeddings_array_compare, speakers)
# normalised_embeddings_egemaps = speaker_normalisation(embeddings_array_egemaps, speakers)

# # Verifying normalised_embeddings_arrays

# models = ['hybrid_byols', 'compare', 'egemaps']
# normalised_embeddings_arrays = {'hybrid_byols': normalised_embeddings_byols, 'compare':normalised_embeddings_compare, 'egemaps':normalised_embeddings_egemaps}

# for model in models:
#   print()
#   print()
#   print('MODEL: {}'.format(model))
#   print()
#   print('The shape of the normalised embeddings array is: {}'.format(normalised_embeddings_arrays[model].shape))
#   print('Normalised Embeddings Array:')
#   print((normalised_embeddings_arrays[model]))
#   print()
#   columnwise_mean = torch.mean(normalised_embeddings_arrays[model], 0)
#   print('Columnwise_mean:')
#   print(columnwise_mean)
#   if torch.all(columnwise_mean < 10**(-6)):
#     print('All means are less than 10**-6')
#   else:
#     print('All means are NOT less than 10**-6')




MODEL: wav2vec2

The shape of the normalised embeddings array is: torch.Size([535, 1024])
Normalised Embeddings Array:
tensor([[-1.0630,  0.8774, -0.3200,  ..., -0.6530,  0.6994,  0.7176],
        [-0.3161,  1.1851, -0.2382,  ..., -0.9840, -1.8121, -0.5755],
        [-0.3934,  0.2471,  0.2027,  ..., -0.9753, -1.1490, -1.0656],
        ...,
        [-0.2436, -0.8298,  2.0756,  ..., -1.1859,  0.9948, -2.2049],
        [ 1.0960,  0.1356,  0.0916,  ...,  0.1838,  0.6432,  0.3809],
        [ 0.4585, -0.0177,  0.4946,  ...,  0.5492,  0.3978,  0.2731]])

Columnwise_mean:
tensor([ 2.2282e-10, -5.3477e-09,  0.0000e+00,  ...,  0.0000e+00,
        -6.2390e-09,  0.0000e+00])
All means are less than 10**-6


MODEL: hubert

The shape of the normalised embeddings array is: torch.Size([535, 1280])
Normalised Embeddings Array:
tensor([[ 0.1948,  1.3250, -1.5221,  ...,  0.4631, -0.3612, -0.1844],
        [-0.5785, -1.7244, -0.6863,  ...,  0.5426,  0.0418, -0.0914],
        [ 1.5018,  0.5686, -0.9444, 

## Train Test splitting functions

In [ ]:
# Defining a function for splitting into train set and test set with diferent speakers in each set

def split_train_test(normalised_embeddings_array, labels, speakers, test_size = 0.30):
  '''
  Splits into training and testing set with different speakers

  Parameters
  ------------
  normalised_embeddings_array: torch tensor
    The tensor containing normalised embeddings 
  labels: list of strings
    The list of emotions corresponding to audio files
  speakers: list 
    The list of speakers

  Returns
  ------------
  X_train: torch tensor
    The normalised embeddings that will be used for training
  X_test: torch tensor
    The normalised embeddings that will be used for testing
  y_train: list
   The labels that will be used for training
  y_test: list
   The labels that will be used for testing

  '''
  # unique speakers in this dataset
  all_speakers = set(speakers)
  # unique speakers in test set
  test_speakers = sample(all_speakers, int(test_size*len(all_speakers)))

  test_speakers_indices = []
  train_speakers_indices = []

  for speaker in all_speakers:
      if speaker in test_speakers:
          speaker_indices = np.where(np.array(speakers)==speaker)[0]
          test_speakers_indices.extend(speaker_indices)
      else:
          speaker_indices = np.where(np.array(speakers)==speaker)[0]
          train_speakers_indices.extend(speaker_indices)

  X_train = normalised_embeddings_array[train_speakers_indices]
  X_test = normalised_embeddings_array[test_speakers_indices]

  y_train = [0 for i in range(len(train_speakers_indices))]
  y_test = [0 for i in range(len(test_speakers_indices))]

  for i,index in enumerate(train_speakers_indices):
      y_train[i] = labels[index]
  for i,index in enumerate(test_speakers_indices):
      y_test[i] = labels[index]

  return X_train, X_test, y_train, y_test


## Train Test splitting on EmoDB

In [ ]:
# Phase_3: Train Test splitting

X_train_wav2vec, X_test_wav2vec, y_train_wav2vec, y_test_wav2vec = split_train_test(normalised_embeddings_wav2vec, labels, speakers, test_size = 0.30)
X_train_hubert, X_test_hubert, y_train_hubert, y_test_hubert = split_train_test(normalised_embeddings_hubert, labels, speakers, test_size = 0.30)

X_trains = {'wav2vec2': X_train_wav2vec, 'hubert': X_train_hubert}
X_tests = {'wav2vec2': X_test_wav2vec, 'hubert': X_test_hubert}
y_trains = {'wav2vec2': y_train_wav2vec, 'hubert': y_train_hubert}
y_tests = {'wav2vec2': y_test_wav2vec, 'hubert': y_test_hubert}

# Verify
models = ['wav2vec2', 'hubert']
for model in models:
  print()
  print()
  print('MODEL: {}'.format(model))
  print()
  print('The shape of X_train is: {}'.format(X_trains[model].shape))
  print('X_train:')
  print(X_trains[model])
  print()
  print('The shape of X_test is: {}'.format(X_tests[model].shape))
  print('X_test:')
  print(X_tests[model])
  print()
  print('The length of y_train is: {}'.format(len(y_trains[model])))
  print('y_train:')
  print(y_trains[model])
  print()
  print('The length of y_test is: {}'.format(len(y_tests[model])))
  print('y_test:')
  print(y_tests[model])


In [ ]:
# # Phase_3: Train Test splitting

# X_train_byols, X_test_byols, y_train_byols, y_test_byols = split_train_test(normalised_embeddings_byols, labels, speakers, test_size = 0.30)
# X_train_compare, X_test_compare, y_train_compare, y_test_compare = split_train_test(normalised_embeddings_compare, labels, speakers, test_size = 0.30)
# X_train_egemaps, X_test_egemaps, y_train_egemaps, y_test_egemaps = split_train_test(normalised_embeddings_egemaps, labels, speakers, test_size = 0.30)

# X_trains = {'hybrid_byols':X_train_byols, 'compare':X_train_compare, 'egemaps':X_train_egemaps}
# X_tests = {'hybrid_byols':X_test_byols, 'compare':X_test_compare, 'egemaps':X_test_egemaps}
# y_trains = {'hybrid_byols':y_train_byols, 'compare':y_train_compare, 'egemaps':y_train_egemaps}
# y_tests = {'hybrid_byols':y_test_byols, 'compare':y_test_compare, 'egemaps':y_test_egemaps}

# # Verify
# models = ['hybrid_byols', 'compare', 'egemaps']
# for model in models:
#   print()
#   print()
#   print('MODEL: {}'.format(model))
#   print()
#   print('The shape of X_train is: {}'.format(X_trains[model].shape))
#   print('X_train:')
#   print(X_trains[model])
#   print()
#   print('The shape of X_test is: {}'.format(X_tests[model].shape))
#   print('X_test:')
#   print(X_tests[model])
#   print()
#   print('The length of y_train is: {}'.format(len(y_trains[model])))
#   print('y_train:')
#   print(y_trains[model])
#   print()
#   print('The length of y_test is: {}'.format(len(y_tests[model])))
#   print('y_test:')
#   print(y_tests[model])




MODEL: wav2vec2

The shape of X_train is: torch.Size([366, 1024])
X_train:
tensor([[-0.3934,  0.2471,  0.2027,  ..., -0.9753, -1.1490, -1.0656],
        [-1.2911, -1.5483, -0.1232,  ...,  0.4996,  0.8120,  0.7581],
        [ 1.1216,  0.4389, -1.3890,  ...,  0.5894, -0.2409,  0.1047],
        ...,
        [ 0.8160,  1.1471, -0.7936,  ...,  0.0416, -1.0520,  0.0961],
        [-1.1908, -1.9220,  1.3309,  ...,  0.3829, -0.1445, -0.1897],
        [-0.4999,  0.2885,  0.4264,  ..., -0.2561,  2.3868,  0.1330]])

The shape of X_test is: torch.Size([169, 1024])
X_test:
tensor([[-2.1918,  0.3515,  1.6224,  ...,  0.1890,  2.4706,  0.3731],
        [ 0.1784,  0.9274, -0.9344,  ...,  0.0177, -0.9814,  1.2774],
        [-1.0864,  1.0387,  0.5590,  ...,  0.3506,  0.0160, -1.2822],
        ...,
        [ 1.4930,  1.7136,  0.2420,  ...,  0.5099, -1.0656,  0.5255],
        [ 1.1636,  0.5158, -0.3835,  ...,  0.7265, -0.0204,  0.4544],
        [-1.3669, -0.8499,  0.2795,  ...,  0.1117, -0.4838,  0.9563]]

## Hyperparameter tuning functions

In [ ]:
# Defining a function for hyperparameter tuning and getting the accuracy on the test set

def get_hyperparams(X_train, X_test, y_train, y_test, classifier, parameters):
  '''
  Splits into training and testing set with different speakers

  Parameters
  ------------
  X_train: torch tensor
    The normalised embeddings that will be used for training
  X_test: torch tensor
    The normalised embeddings that will be used for testing
  y_train: list
    The labels that will be used for training
  y_test: list
    The labels that will be used for testing
  classifier: object
    The instance of the classification model 
  parameters: dictionary
    The dictionary of parameters for GridSearchCV 

  Returns
  ------------
    The dictionary of the best hyperparameters
  
  '''
  grid = GridSearchCV(classifier, param_grid = parameters, cv=5, scoring='recall_macro')                     
  grid.fit(X_train,y_train)
  print('Accuracy :',grid.best_score_)
  print('Best Parameters: {}'.format(grid.best_params_))
  print('Accuracy on test_set: {}'.format(grid.score(X_test, y_test)))
  return grid.score(X_test, y_test)


## Hyperparameter tuning and getting accuracy on EmoDB

In [1]:
models = ['wav2vec2', 'hubert']

# Logistic Regression
print('Logistic Regression:')
classifier = LogisticRegression()
parameters = {'penalty' : ['l1','l2'], 'C': np.logspace(-4,2,7), 'solver': ['newton-cg', 'lbfgs', 'liblinear']}
for model in models:
  print('MODEL: {}'.format(model))
  get_hyperparams(X_trains[model], X_tests[model], y_trains[model], y_tests[model], classifier, parameters)
print()

# Support Vector Machine
print('Support Vector Machine:')
classifier = SVC()
parameters = {'C': np.logspace(-2,3,6), 'gamma': np.logspace(-5,2,8), 'kernel':['rbf','poly','sigmoid','linear']}
for model in models:
  print('MODEL: {}'.format(model))
  get_hyperparams(X_trains[model], X_tests[model], y_trains[model], y_tests[model], classifier, parameters)
print()

# Random Forest Classifier
print('Random Forest Classifier:')
classifier = RandomForestClassifier()
parameters = {'n_estimators' : [50,100,200], 'max_features' : ['auto', 'log2', 'sqrt'], 'bootstrap' : [True, False]}
for model in models:
  print('MODEL: {}'.format(model))
  get_hyperparams(X_trains[model], X_tests[model], y_trains[model], y_tests[model], classifier, parameters)
print()


In [ ]:
# models = ['hybrid_byols', 'compare', 'egemaps']

# # Logistic Regression
# print('Logistic Regression:')
# classifier = LogisticRegression()
# parameters = {'penalty' : ['l1','l2'], 'C': np.logspace(-4,2,7), 'solver': ['newton-cg', 'lbfgs', 'liblinear']}
# for model in models:
#   print('MODEL: {}'.format(model))
#   get_hyperparams(X_trains[model], X_tests[model], y_trains[model], y_tests[model], classifier, parameters)
# print()

# # Support Vector Machine
# print('Support Vector Machine:')
# classifier = SVC()
# parameters = {'C': np.logspace(-2,3,6), 'gamma': np.logspace(-5,2,8), 'kernel':['rbf','poly','sigmoid','linear']}
# for model in models:
#   print('MODEL: {}'.format(model))
#   get_hyperparams(X_trains[model], X_tests[model], y_trains[model], y_tests[model], classifier, parameters)
# print()

# # Random Forest Classifier
# print('Random Forest Classifier:')
# classifier = RandomForestClassifier()
# parameters = {'n_estimators' : [50,100,200], 'max_features' : ['auto', 'log2', 'sqrt'], 'bootstrap' : [True, False]}
# for model in models:
#   print('MODEL: {}'.format(model))
#   get_hyperparams(X_trains[model], X_tests[model], y_trains[model], y_tests[model], classifier, parameters)
# print()


# Results

In [3]:
# results = {'Logistic Regression': {'wav2vec2': 0, 'hubert': 0, 'hybrid_byols': 0, 'compare': 0, 'egemaps': 0},
#         'Support Vector Machine': {'wav2vec2': 0, 'hubert': 0, 'hybrid_byols': 0, 'compare': 0, 'egemaps': 0},
#         'Random Forest Classification': {'wav2vec2': 0, 'hubert': 0, 'hybrid_byols': 0, 'compare': 0, 'egemaps': 0}}

In [2]:
# import pandas as pd

# df = pd.DataFrame(results)
# df